In [41]:
import tempfile
import os
import uuid
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
import logging 
import PyPDF2
import numpy as np
import os
os.environ['CURL_CA_BUNDLE'] = ''
import warnings
warnings.filterwarnings("ignore")

In [2]:
llm = GigaChat(credentials='YzYxYTQwYTgtZGE4ZC00NWEyLWFiOGEtZmQzNzExZDg1ZWQzOmZkMTg1OWE1LWM2YTAtNDNiNy05YzAwLTg0NjE3YWE0YmE4Mw==', 
               verify_ssl_certs=False)

In [42]:
def get_embeddings():
    model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    model_kwargs = {'device': 'cpu'}
    # model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': False}
    return HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
        )

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyMuPDFLoader

documents = []
for file in os.listdir('files'):
    if file.endswith('.pdf'):
        pdf_path = './files/' + file
        loader = PyMuPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = './files/' + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = './files/' + file
        loader = TextLoader(text_path, encoding='utf-8')
        documents.extend(loader.load())

In [7]:
%%time
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,)
splits = text_splitter.split_documents(documents)
embedding = get_embeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

CPU times: total: 38.1 s
Wall time: 11 s


In [8]:
vectorstore.save_local(folder_path='db/')

In [12]:
my_vector = FAISS.load_local(folder_path='db/', embeddings=get_embeddings(), allow_dangerous_deserialization=True)
my_vector
print(vectorstore)
print(my_vector)

In [6]:
with open('files/chroma.sqlite3') as f:
    print(f)

<_io.TextIOWrapper name='files/chroma.sqlite3' mode='r' encoding='cp1251'>


In [13]:
file_names = []
for file in os.listdir('files'):
    path = './files/' + file
    file_names.append(os.path.basename(path))
print(file_names)

['Михнин_А_Е_Логистическая_регрессия_2023.pdf']


In [44]:
import pandas as pd
df = pd.read_pickle('ru_rag_test_dataset.pkl')
df.head(100)

,Вопрос,Правильный ответ,Контекст,Файл
0,Что может вызвать цунами?,Землетрясение,"Землетрясения, извержения вулканов и другие по...",19985.txt
1,Кто написал роман «Хижина дяди Тома»?,Г. Бичер-Стоу,«Хижина дяди Тома» (англ. Uncle Tom's Cabin) —...,273921.txt
2,Кто создал Зимний дворец в Санкт-Петербурге?,Растрелли,Зимний дворец (в 1918—1943 годах — Дворец Иску...,50.txt
3,Какое современное государство занимает террито...,Ирак,"Нине́ви́я или Нин (аккад. 𒌷𒉌𒉡𒀀, Ninwe; ассир. ...",36300.txt
4,Как звали отца Северуса Снейпа?,Тоббиас Снегг (Снейп),Происхождение Снегга в основном стало известно...,172300.txt
...,...,...,...,...
95,Гражданином какой страны был открыватель пениц...,Великобритания.,Сэр Александр Флеминг (англ. Alexander Fleming...,82171.txt
96,Кто написал оперу Евгений Онегин?,П. И. Чайковский,"Евге́ний Оне́гин — лирические сцены в 3 актах,...",185772.txt
97,Какой город является центром Уральского федера...,Екатеринбург,Административный центр и крупнейший город — Ек...,5362.txt
98,Кинематографисты какой страны сняли знаменитый...,ПНР (Польская народная республика),"«Ставка больше, чем жизнь» (польск. Stawka wię...",1813030.txt


In [73]:
documents[0].metadata['source'][8::]

'100141.txt'

In [74]:
file_names = []
for document in documents[:100]:
    file_names.append(document.metadata['source'][8::])
file_names

['100141.txt',
 '101.txt',
 '101560.txt',
 '10222.txt',
 '10223.txt',
 '10292.txt',
 '10300.txt',
 '103043.txt',
 '1032522.txt',
 '103353.txt',
 '1038.txt',
 '1062192.txt',
 '10647.txt',
 '10658.txt',
 '10669.txt',
 '10670.txt',
 '106717.txt',
 '1068248.txt',
 '1068936.txt',
 '1069005.txt',
 '106970.txt',
 '107225.txt',
 '107304.txt',
 '107694.txt',
 '107723.txt',
 '1077294.txt',
 '107928.txt',
 '108207.txt',
 '108301.txt',
 '1084.txt',
 '109038.txt',
 '109211.txt',
 '10947.txt',
 '1097526.txt',
 '110.txt',
 '11028.txt',
 '110316.txt',
 '110520.txt',
 '111255.txt',
 '111393.txt',
 '111408.txt',
 '1118275.txt',
 '1121.txt',
 '1125611.txt',
 '11314.txt',
 '11377.txt',
 '11378.txt',
 '113796.txt',
 '1142251.txt',
 '11514.txt',
 '11525.txt',
 '11542.txt',
 '118103.txt',
 '118505.txt',
 '11987.txt',
 '1210101.txt',
 '121720.txt',
 '122720.txt',
 '122991.txt',
 '1234185.txt',
 '124402.txt',
 '1247934.txt',
 '1248659.txt',
 '124940.txt',
 '1261274.txt',
 '126868.txt',
 '1275460.txt',
 '128351

In [75]:
ds = df[df['Файл'].isin(file_names)]

In [80]:
ds['Вопрос'].iloc[0]

'В какой стране находится гора Эверест?'

In [27]:
contextualize_q_system_prompt = (
       "Запоминай историю чата и последний вопрос пользвоателя"
    "Сначала пытайся ответить на вопрос, основываясь на истории чата"
    "сформулируй отдельный вопрос без истории чата"
    "Не отвечай на этот вопрос, просто перефразируй его, если это необходимо, в противном случае верни как есть"
    )
contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    ### Answer question ###
system_prompt = (
       ''' 
    Ты подробно и структурировоно отвечаешь на вопрос на основе загруженных статей об искуственном интелекте.
    Предлагай пользователю задать дополнительные похожие вопросы. Предлагайте контекстную и справочную информацию,
    основываясь на метаданнтых каждого документа из контекста.
    Если ответа на вопрос нет в контексте, предложи вопрос, ответ на который есть в контексте
    Контекст: {context}
    Ответ:'''
    )
qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    ### Statefully manage chat history ###
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )


In [37]:
conversational_rag_chain.get_session_history('abc123')

InMemoryChatMessageHistory(messages=[HumanMessage(content='ПРивет', additional_kwargs={}, response_metadata={}), AIMessage(content='Здравствуйте! Я здесь, чтобы помочь вам найти ответы на ваши вопросы о логистической регрессии и искусственном интеллекте. Если у вас есть какие-либо вопросы или нужна дополнительная информация, пожалуйста, спрашивайте.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ПРивет', additional_kwargs={}, response_metadata={}), AIMessage(content='Здравствуйте! Я здесь, чтобы помочь вам найти ответы на ваши вопросы о логистической регрессии и искусственном интеллекте. Если у вас есть какие-либо вопросы или нужна дополнительная информация, пожалуйста, спрашивайте.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ПРивет', additional_kwargs={}, response_metadata={}), AIMessage(content='Здравствуйте! Я здесь, чтобы помочь вам найти ответы на ваши вопросы о логистической регрессии и искусственном интеллекте. Если у вас есть какие

In [36]:
conversational_rag_chain.invoke({'input': 'ПРивет'}, config={
                        "configurable": {"session_id": 'abc123'}
                    })

{'input': 'ПРивет',
 'chat_history': [HumanMessage(content='ПРивет', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Здравствуйте! Я здесь, чтобы помочь вам найти ответы на ваши вопросы о логистической регрессии и искусственном интеллекте. Если у вас есть какие-либо вопросы или нужна дополнительная информация, пожалуйста, спрашивайте.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='ПРивет', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Здравствуйте! Я здесь, чтобы помочь вам найти ответы на ваши вопросы о логистической регрессии и искусственном интеллекте. Если у вас есть какие-либо вопросы или нужна дополнительная информация, пожалуйста, спрашивайте.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='ПРивет', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Здравствуйте! Я здесь, чтобы помочь вам найти ответы на ваши вопросы о логистической регрессии и искусственном интеллекте. Если у вас

In [97]:
ds['Правильный ответ'].iloc[1]

'Винер'

In [90]:
import re
import json
from rouge import Rouge
from pymorphy3 import MorphAnalyzer

f = open('stopwords-ru.json', encoding='utf-8')
stop_words = json.load(f)
#print(stop_words)

morph = MorphAnalyzer()
patterns = "[«»°!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def lemmatize(string):
    clear = re.sub(patterns, ' ', string)
    tokens = []
    for token in clear.split():
        if token:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            if token not in stop_words:
                tokens.append(token)
    tokens = ' '.join(tokens)
    return tokens


In [91]:
def get_llm_score(answer, answer_true):
    answer = lemmatize(answer)
    answer_true = lemmatize(answer_true)
    if len(answer) == 0:
        answer = '-'
        
    rouge = Rouge()
    scores = rouge.get_scores(answer, answer_true)[0]
    rouge_1 = round(scores['rouge-1']['r']*100, 2)
    
    return rouge_1

In [99]:
get_llm_score(conversational_rag_chain.invoke({'input': ds['Вопрос'].iloc[0]}, config={
                        "configurable": {"session_id": 'abc123'}
                    })['answer'], ds['Правильный ответ'].iloc[0])

100.0

In [105]:
full_score = []

In [121]:
for i in range(len(ds)):
    full_score.append(get_llm_score(conversational_rag_chain.invoke({'input': ds['Вопрос'].iloc[i]}, config={
                        "configurable": {"session_id": 'abc123'}
                    })['answer'], ds['Правильный ответ'].iloc[i]))

Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist


ResponseError: (URL('https://gigachat.devices.sberbank.ru/api/v1/chat/completions'), 402, b'{"status":402,"message":"Payment Required"}\n', Headers([('server', 'nginx'), ('date', 'Wed, 18 Sep 2024 08:28:46 GMT'), ('content-type', 'application/json; charset=utf-8'), ('content-length', '44'), ('connection', 'keep-alive'), ('access-control-allow-credentials', 'true'), ('access-control-allow-headers', 'Origin, X-Requested-With, Content-Type, Accept, Authorization'), ('access-control-allow-methods', 'GET, POST, DELETE, OPTIONS'), ('access-control-allow-origin', 'https://beta.saluteai.sberdevices.ru'), ('x-request-id', '5698276d-1247-4dc2-a418-c3de36dcbd87'), ('x-session-id', 'a732b1aa-b129-4d20-a5a2-c4f4332d1f86'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains')]))

In [117]:
conversational_rag_chain.invoke({'input': ds['Вопрос'].iloc[-3]}, config={
                        "configurable": {"session_id": 'abc123'}
                    })['answer']

'Невская битва произошла в 1240 году.'

In [118]:
ds['Правильный ответ'].iloc[-3]

'15 июля 1240 г'

In [122]:
len(full_score)

122

In [123]:
a = np.asarray(full_score[90::])

In [127]:
a

array([100.  , 100.  , 100.  , 100.  , 100.  , 100.  , 100.  , 100.  ,
         0.  ,   0.  , 100.  , 100.  ,  66.67,   0.  , 100.  , 100.  ,
         0.  , 100.  , 100.  , 100.  , 100.  ,  33.33, 100.  , 100.  ,
       100.  , 100.  , 100.  ,   0.  , 100.  , 100.  ,   0.  ,   0.  ])

In [119]:
ds['Правильный ответ']

7                         Непал
30                        Винер
45                    Агамемнон
49                     Тюрингия
58                  Теночтитлан
                 ...           
845                           6
856                      1938 г
862              15 июля 1240 г
877    Владимиром Святославичем
915            Герхард фон Риле
Name: Правильный ответ, Length: 112, dtype: object

In [6]:
from g4f.client import Client

client = Client()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Ты кто"}]
)
print(response.choices[0].message.content)


Привет! Я искусственный интеллект, готов помочь вам с вашими вопросами или задачами. Чем могу помочь?


In [29]:
from g4f.client import Client
from g4f.Provider import BingCreateImages, OpenaiChat, Gemini, Bing, Free2GPT

client = Client(
    provider=Free2GPT
)

In [37]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Что ты умеешь?"}],
)

print(response.choices[0].message.content)

Я - это языковая модель, и я умею выполнять множество задач! Вот некоторые из них:

1. **Отвечать на вопросы**: Я могу отвечать на вопросы на различные темы, от науки и истории до культуры и развлечений.
2. **Помогать с языками**: Я могу помочь с грамматикой, лексикой и синтаксисом многих языков, включая русский, английский, испанский, французский и другие.
3. **Составлять тексты**: Я могу генерировать тексты на заданную тему, от коротких ответов до длинных статей.
4. **Переводить тексты**: Я могу переводить тексты с одного языка на другой, включая русский, английский, испанский, французский и другие.
5. **Помогать с задачами**: Я могу помочь с решением математических задач, от простых арифметических операций до сложных задач по математике и физике.
6. **Играть в игры**: Я могу играть в текстовые игры, такие как "20 вопросов", "Виселица" и другие.
7. **Генерировать идеи**: Я могу генерировать идеи для написания статей, рассказов, стихов и других творческих работ.
8. **Помогать с написа

In [40]:
llm = GigaChat(credentials='YzYxYTQwYTgtZGE4ZC00NWEyLWFiOGEtZmQzNzExZDg1ZWQzOmZkMTg1OWE1LWM2YTAtNDNiNy05YzAwLTg0NjE3YWE0YmE4Mw==', 
               verify_ssl_certs=False)

In [ ]:
contextualize_q_system_prompt = (
       "Запоминай историю чата и последний вопрос пользвоателя"
    "Сначала пытайся ответить на вопрос, основываясь на истории чата"
    "сформулируй отдельный вопрос без истории чата"
    "Не отвечай на этот вопрос, просто перефразируй его, если это необходимо, в противном случае верни как есть"
    )
contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    ### Answer question ###
system_prompt = (
       ''' 
    Ты подробно и структурировоно отвечаешь на вопрос на основе загруженных статей об искуственном интелекте.
    Предлагай пользователю задать дополнительные похожие вопросы. Предлагайте контекстную и справочную информацию,
    основываясь на метаданнтых каждого документа из контекста.
    Если ответа на вопрос нет в контексте, предложи вопрос, ответ на который есть в контексте
    Контекст: {context}
    Ответ:'''
    )
qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    ### Statefully manage chat history ###
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
